In [0]:
# For colab
!pip install https://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-win_amd64.whl
!pip install torchvision

In [2]:
# If return 'True', GPU is available on the current environment
import torch
print('Use Cuda:', torch.cuda.is_available())
print('#GPUs:', torch.cuda.device_count())

Use Cuda: True
#GPUs: 1


In [3]:
# Run script-based by copying this cell to .py
# Based from https://github.com/pytorch/examples/blob/master/mnist/main.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

class myArgs():
  def __init__(self):
    self.batch_size = 64         # Training batch size
    self.test_batch_size = 1000  # Test batch size
    self.epochs = 10             # Epochs
    self.lr = 0.01               # Learning rate
    self.no_cuda = False         # Force to not use cuda(GPU)
    self.seed = 1                # Random seed
    self.log_interval = 0        # Log progress(#batch interval) during training
    self.save_model = False      # Save model after finish training

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if args.log_interval > 0 and batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    args = myArgs()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)


    print('Use GPU:', use_cuda)
    model = Net()
    if use_cuda:
      print('#GPUs:', torch.cuda.device_count())
      if torch.cuda.device_count() > 1:
          model = nn.DataParallel(model)
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr)

    for epoch in range(1, args.epochs + 1):
        print('==== Epoch {} ===='.format(epoch))
        ts = time.time()
        train(args, model, device, train_loader, optimizer, epoch)
        print('Training time: {} s'.format(time.time()-ts))

        ts = time.time()
        test(args, model, device, test_loader)
        print('Test time: {} s'.format(time.time()-ts))

        print()

    if (args.save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")
        
if __name__ == '__main__':
    ts_all = time.time()
    main()
    print('\n==== Summary====')
    print('Total process time: {:.6f} s'.format(time.time()-ts_all))


Use GPU: True
#GPUs: 1
==== Epoch 1 ====
Training time: 15.230961561203003 s
Test set: Average loss: 0.1675, Accuracy: 9501/10000 (95%)
Test time: 1.9964420795440674 s

==== Epoch 2 ====
Training time: 15.360131978988647 s
Test set: Average loss: 0.0999, Accuracy: 9707/10000 (97%)
Test time: 1.978642463684082 s

==== Epoch 3 ====
Training time: 15.384657621383667 s
Test set: Average loss: 0.0724, Accuracy: 9774/10000 (98%)
Test time: 1.9669594764709473 s

==== Epoch 4 ====
Training time: 15.373796463012695 s
Test set: Average loss: 0.0581, Accuracy: 9822/10000 (98%)
Test time: 1.9704499244689941 s

==== Epoch 5 ====
Training time: 15.47372055053711 s
Test set: Average loss: 0.0614, Accuracy: 9810/10000 (98%)
Test time: 1.942704439163208 s

==== Epoch 6 ====
Training time: 15.282972812652588 s
Test set: Average loss: 0.0450, Accuracy: 9852/10000 (99%)
Test time: 1.978839635848999 s

==== Epoch 7 ====
Training time: 15.22230577468872 s
Test set: Average loss: 0.0436, Accuracy: 9849/10000